# Notebook : Génération d'Images, Low-Code AI, Function Calling, RAG

Dans ce notebook, nous allons tour à tour découvrir :

1. Comment générer des **images** à partir de prompts en texte (ex: DALL-E, Midjourney).
2. Comment créer des **applications low-code** enrichies par l'IA, grâce à **Power Platform** (Copilot, AI Builder).
3. **Function Calling** côté OpenAI : structurer les réponses d’un LLM pour déclencher des actions.
4. **RAG** (Retrieval Augmented Generation) et **bases vectorielles** (indexation, recherche sémantique, chunking, etc.).



## Prérequis & Installation

- **Python 3.9+** (ou version ultérieure).
- Un compte [OpenAI](https://platform.openai.com/) et une clé d’API valide.
- Le fichier `.env` contenant votre clé d’API :
OPENAI_API_KEY=sk-...



### Pourquoi la génération d'images via IA ?

Les modèles de génération d'images, tels que **DALL-E** (OpenAI) ou **Midjourney**, ont la capacité de créer des visuels originaux à partir de simples descriptions textuelles (prompts).  
- **Applications concrètes** :  
  - Design rapide de prototypes (marketing, publicité)  
  - Création artistique (concept art, storyboards)  
  - Illustrations pédagogiques ou infographiques  
- **Limitations** :  
  - Les images peuvent contenir des incohérences (proportions bizarres, doigts supplémentaires, etc.)  
  - Certaines requêtes contraires aux politiques d’utilisation peuvent être bloquées  

Dans la suite, nous allons voir comment **OpenAI** gère la génération d’images via l’API `images.generate()`, et comment intégrer ces visuels dans un flux d'automatisation (Low-Code) ou dans des applications web.


In [ ]:
# ============================
# Installation
# ============================

%pip install openai tiktoken python-dotenv
# Remarque : Aucune fin de ligne en commentaire pour éviter l'erreur


In [ ]:
import os
import requests
# from PIL import Image
from IPython.display import Image
from IPython.core.display import HTML 
import openai
from dotenv import load_dotenv

# Charger le fichier .env pour la clé OPENAI_API_KEY
load_dotenv()

# Config globale
openai.api_key = os.getenv("OPENAI_API_KEY")

try:
    response = openai.images.generate(
        prompt="Lapin sur un cheval tenant une sucette, dans un champ brumeux"
    )
    # response est un ImagesResponse
    image_url = response.data[0].url
    print("Image URL:", image_url)

    # Téléchargement de l'image
    # img_data = requests.get(image_url).content
    # with open("my_image.png","wb") as f:
    #     f.write(img_data)

    # # Ouverture
    # img = Image.open("my_image.png")
    # img.show()
    
    
    image = Image(url= image_url)
    display(image)

except openai.APIConnectionError as e:
    print("Erreur de connexion réseau:", e)
except openai.RateLimitError as e:
    print("Limite atteinte ou quota dépassé:", e)
except openai.APIStatusError as e:
    print("Erreur HTTP renvoyée par l'API (4xx, 5xx, etc.):", e)
except openai.APIError as e:
    print("Autre erreur OpenAI:", e)


In [ ]:
# ============================
#  Comparaison de prompts d'images
# ============================

image_prompts = [
    "A small kitten wearing a hat, cartoon style",
    "A realistic portrait of a small kitten wearing a cowboy hat in the desert",
    "A small kitten wearing a futuristic helmet in cyberpunk style, neon colors"
]

for i, prompt in enumerate(image_prompts):
    try:
        print(f"--- Prompt #{i+1}: {prompt} ---")
        response_img = openai.images.generate(prompt=prompt)
        img_url = response_img.data[0].url
        print("Image URL:", img_url)
        # Optionnel : display() si tu es dans un environnement Jupyter
    except Exception as e:
        print("⚠️ Erreur lors de la génération d'image:", e)
    print()


## 1.2 Méta-prompts et usage responsable

Pour gérer un usage plus responsable et filtrer des images non souhaitées, on peut ajouter un 
**meta-prompt** en amont, décrivant les restrictions (ex: Safe for Work, No adult content, etc.).

Ex:
You are an assistant that only generates children-friendly images. [... consignes ...]


# 2. Low-Code AI Apps (Power Platform)

## 2.1 Introduction
Power Platform inclut :
- Power Apps (construction rapide d'apps)
- Power Automate (workflows et automatisations)
- Dataverse (stockage de données)
- AI Builder (modèles IA pré-construits)
- Copilot (assistant pour générer tables, flux, e-mails)

Avantages : construction **no-code / low-code** pour mettre en place des solutions rapidement, 
y compris connectées à des services IA.


## 2.2 Copilot dans Power Apps : Student Assignment Tracker

Exemple : On veut un **Student Assignment Tracker**.

1. Sur la home de [Power Apps](https://make.powerapps.com), on saisit dans la zone Copilot : 
   "I want an app to track and manage student assignments."
2. Copilot propose une table Dataverse (champs Title, DateDue, StudentName, etc.)
3. Personnaliser la table (ajouter `StudentEmail`, etc.)
4. Cliquer "Create app" => Copilot génère une **Canvas App** auto.
5. Ajouter une page (screen) pour "Envoyer un email" (Prompt : "I want a screen to send an email to the student").

On obtient en quelques clics un début d'application.


## 2.3 Copilot dans Power Automate : Invoice Processing

Même concept : Dans [Power Automate](https://make.powerautomate.com),
on demande "Process an invoice when it arrives in my mailbox", 
Copilot propose un flux (trigger: new mail arrives + extractions + email)...

On peut ensuite y intégrer **AI Builder** : 
- ex: le prébuilt model "Invoice Processing" pour extraire `supplier`, `amount`, etc.
- stocker dans Dataverse, 
- email de confirmation.

C’est un gros gain de temps pour la finance ou la logistique !


# 3. Function Calling (OpenAI)


## 3.1 Pourquoi ?

Sans function calling, le LLM renvoie du texte non structuré. 
Difficile d’automatiser (ex: parse JSON, exécuter une fonction tierce).
Avec function calling, on déclare un `schema` JSON, 
le LLM répond par un `function_call`: 
- Nom de la fonction 
- Arguments structurés

Ensuite on exécute la fonction en Python (ou autre).


In [ ]:
import openai

messages = [
    {"role": "user", "content": "Find me a good course for a beginner developer to learn Azure."}
]

functions = [
  {
    "name": "search_courses",
    "description": "Retrieves relevant courses based on role, product & level",
    "parameters": {
      "type": "object",
      "properties": {
        "role":   {"type":"string","description":"the role of the user"},
        "product":{"type":"string","description":"the product/tech"},
        "level": {"type":"string","description":"the user skill level"}
      },
      "required": ["role","product","level"]
    }
  }
]


try:
    response = openai.chat.completions.create(
        model="gpt-4o-mini",  
        messages=messages,
        functions=functions,
        function_call="auto"  # laisse le LLM décider s’il appelle la fonction
    )

    print("Réponse brute:\n", response.choices[0].message)

except openai.RateLimitError as e:
    print("Limite atteinte:", e)
except openai.APIError as e:
    print("Autre erreur:", e)


In [ ]:
import json


def search_courses(role,product,level):
    # Ton code Python => Appel API Microsoft Learn
    # On renvoie un JSON/string
    return "Liste de cours: Azure Fundamentals, etc."

resp_msg = response.choices[0].message
if resp_msg.function_call:
    fn_name = resp_msg.function_call.name
    fn_args = json.loads(resp_msg.function_call.arguments)
    
    # Exécuter la fonction Python correspondante
    result = search_courses(**fn_args)

    # On crée deux messages :
    # 1) le function_call
    # 2) le role="function" + content du résultat
    second_messages = [
      {"role":"assistant","function_call": {"name":fn_name,"arguments":resp_msg.function_call.arguments}},
      {"role":"function","name":fn_name,"content": result}
    ]

    # On relance le chat
    final_resp = openai.chat.completions.create(
       model="gpt-4o-mini",
       messages=messages + second_messages
    )
    print("Réponse finale:\n", final_resp.choices[0].message.content)


## Function Calling Avancé avec OpenAI : Chaînage de Fonctions  

Ce bloc de code illustre un exemple avancé de **Function Calling** avec OpenAI, permettant à l'IA d'orchestrer plusieurs appels de fonctions de manière autonome.  

### 🛠 Fonctionnalités mises en œuvre :  
1. **Premier appel à l'API OpenAI** :  
   - L'utilisateur demande de planifier une réunion.  
   - L'IA détecte qu'il faut appeler `create_meeting_event` et génère les arguments nécessaires (`topic`, `date`, `participants`).  
   - Le modèle ne renvoie pas de texte brut, mais un `function_call` contenant les paramètres de la réunion.  

2. **Exécution locale de `create_meeting_event` en Python** :  
   - La fonction génère un objet événement fictif avec un `event_id`.  
   - Ce résultat est transmis à l'IA en tant que réponse fonctionnelle (`role="function"`).  

3. **Deuxième appel à l'API OpenAI** :  
   - Sur la base de l'événement créé, l'IA décide de générer un email de confirmation en appelant `send_email`.  
   - L'IA fournit les arguments (`subject`, `body`, `recipients`).  

4. **Exécution locale de `send_email` en Python** :  
   - Simulation de l'envoi d'email avec un affichage console.  
   - Message de confirmation `"Email sent successfully!"`.  

### 📌 Preuve du bon fonctionnement :  
✅ **Chaînage réussi** : OpenAI a déclenché **deux appels de fonction distincts**, prouvant la capacité du modèle à raisonner sur plusieurs étapes.  
✅ **Exécution hybride** : L'IA décide des actions à effectuer, mais l'exécution est déléguée au code Python.  
✅ **Application possible** : Ce workflow peut être adapté pour intégrer des bases de données, envoyer de vrais emails ou automatiser des tâches complexes.  

🔹 **Exemple d'affichage console :**  
```
Raw response: ChatCompletionMessage(..., function_call=FunctionCall(...))
=== Simulated Email ===
Subject: Confirmation de la réunion sur l'état du projet
To: ['Alice', 'Bob']
Body: Bonjour Alice et Bob,
Je vous confirme que la réunion sur l'état du projet est planifiée pour mardi prochain, le 31 octobre 2023, à 10h.
À bientôt,
L'équipe de gestion de projet
Email function result: Email sent successfully!
```

In [ ]:
# ============================
# Function Calling (avancé) avec 2 fonctions
# ============================

import json
import openai

def create_meeting_event(topic, date, participants):
    # Exemple simulé : création d'un objet event
    return {
        "event_id": "evt-001",
        "topic": topic,
        "date": date,
        "participants": participants
    }

def send_email(subject, body, recipients):
    # Exemple simulé : envoi d'un email
    print("=== Simulated Email ===")
    print("Subject:", subject)
    print("To:", recipients)
    print("Body:", body)
    return "Email sent successfully!"

functions = [
    {
        "name": "create_meeting_event",
        "description": "Créer un événement de réunion",
        "parameters": {
            "type": "object",
            "properties": {
                "topic": {"type": "string"},
                "date": {"type": "string"},
                "participants": {
                    "type": "array",
                    "items": {"type": "string"}
                }
            },
            "required": ["topic", "date", "participants"]
        }
    },
    {
        "name": "send_email",
        "description": "Envoyer un email",
        "parameters": {
            "type": "object",
            "properties": {
                "subject": {"type": "string"},
                "body": {"type": "string"},
                "recipients": {
                    "type": "array",
                    "items": {"type": "string"}
                }
            },
            "required": ["subject", "body", "recipients"]
        }
    }
]

messages_fc = [
    {
        "role": "user",
        "content": (
            "Planifie une réunion sur l'état du projet pour mardi prochain à 10h "
            "avec Alice et Bob, puis envoie un email de confirmation."
        )
    }
]

# L'appel initial : le modèle peut décider d'appeler
# create_meeting_event, send_email, ou rien (function_call="auto").
response_fc = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages_fc,
    functions=functions,
    function_call="auto"
)

assistant_msg = response_fc.choices[0].message
print("Raw response:", assistant_msg)

# Vérifier si le modèle appelle une fonction
if assistant_msg.function_call:
    fn_name = assistant_msg.function_call.name
    fn_args_str = assistant_msg.function_call.arguments  # chaîne JSON
    fn_args = json.loads(fn_args_str)

    if fn_name == "create_meeting_event":
        # 1) Exécuter la fonction create_meeting_event côté Python
        result_event = create_meeting_event(**fn_args)

        # 2) Créer de nouveaux messages (assistant + function)
        second_messages = messages_fc + [
            {
                "role": "assistant",
                "function_call": {
                    "name": fn_name,
                    "arguments": json.dumps(fn_args)
                }
            },
            {
                "role": "function",
                "name": fn_name,
                "content": json.dumps(result_event)
            }
        ]

        # 3) Relancer le chat pour voir si le modèle appelle la 2e fonction
        second_response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=second_messages,
            functions=functions,
            function_call="auto"
        )
        second_msg = second_response.choices[0].message

        if second_msg.function_call:
            fn2_name = second_msg.function_call.name
            fn2_args_str = second_msg.function_call.arguments
            fn2_args = json.loads(fn2_args_str)

            if fn2_name == "send_email":
                # Exécution de la seconde fonction
                result_email = send_email(**fn2_args)
                print("Email function result:", result_email)
            else:
                print(f"The model called a different function: {fn2_name}")
        else:
            print("No second function call was triggered.")
    else:
        print(f"The model called a different function: {fn_name}")
else:
    print("No function call triggered by the assistant.")


# 4. Retrieval Augmented Generation & Vector Databases

## 4.1 Principe
Un LLM (ex: GPT) a une limite : il ne connaît pas forcément nos documents internes. 
RAG => on stocke nos docs dans une base vectorielle (embeddings), 
puis à chaque question, on envoie au LLM les passages pertinents (retrieval + augmentation).

## 4.2 Création d’une base vectorielle

- On découpe (chunk) nos documents en petits segments (ex: 400 tokens).
- On calcule embeddings (ex: text-embedding-ada-002).
- On stocke : ex. Cosmos DB, Pinecone, ChromaDB, Elasticsearch, Qdrant, etc.

In [ ]:
%pip install scikit-learn numpy pandas requests beautifulsoup4 lxml


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL du débat
url = "https://home.nps.gov/liho/learn/historyculture/debate1.htm"

# Requête HTTP
response = requests.get(url)
html = response.text  # contenu HTML sous forme de string

# print(html)

# On parse avec BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Sélection du noeud principal.
# Selon ton info: div.ColumnMain:nth-child(2)
# (Le "nth-child(2)" est parfois incertain, on peut tenter un select plus large.)
main_div = soup.select_one("div.ColumnMain")

if not main_div:
    raise ValueError("Impossible de trouver le div.ColumnMain dans la page !")

# Extraction du texte brut (on sépare par " " pour éviter les collisions)
debate_text = main_div.get_text(separator="\n", strip=True)

#Enregistrement du texte dans un fichier
with open("debate.txt", "w") as f:
    f.write(debate_text)






In [ ]:
# Chargement depuis le fichier
with open("debate.txt", "r") as f:
    debate_text = f.read()

In [ ]:
# Pour débogage:
print("=== Longueur du texte récupéré:", len(debate_text))
print(debate_text[:10000], "...")

In [ ]:
df = pd.DataFrame(
    [
        {
            "title":"First Debate: Ottawa, Illinois (NPS)",
            "text": debate_text
        }
    ]
)
df


In [ ]:
def split_text_into_chunks(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk_words = words[start:end]
        chunk = " ".join(chunk_words)
        chunks.append(chunk)
        start += (chunk_size - overlap)
    return chunks

rows = []
for _, row in df.iterrows():
    splitted = split_text_into_chunks(row["text"], chunk_size=400, overlap=50)
    for chunk in splitted:
        rows.append({
            "title": row["title"],
            "chunk": chunk
        })

df_chunks = pd.DataFrame(rows)
print("Nombre de chunks =", len(df_chunks))
df_chunks.head()


In [ ]:
from openai import OpenAI

# Initialiser le client OpenAI (indispensable avec la nouvelle API)
client = OpenAI()

def create_embedding(text: str):
    try:
        response = client.embeddings.create(
            model="text-embedding-3-large",
            input=[text]  # ⚠️ Doit être une **liste**
        )
        return response.data[0].embedding  # Extraction correcte

    except Exception as e:
        print(f"⚠️ Erreur lors de la génération d'embedding: {e}")
        return None


In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Générer les embeddings pour les chunks du DataFrame
df_chunks["embedding"] = df_chunks["chunk"].apply(create_embedding)


In [ ]:

# Nettoyage des embeddings
all_vectors = np.array([emb for emb in df_chunks["embedding"] if emb is not None])  # Exclure None
nn = NearestNeighbors(n_neighbors=3, metric="euclidean")
nn.fit(all_vectors)

def retrieve(user_query: str) -> str:
    try:
        # Générer l'embedding de la requête
        q_emb = create_embedding(user_query)
        if q_emb is None:
            return "⚠️ Impossible de générer un embedding pour la requête."

        dist, idx = nn.kneighbors([q_emb])

        # Récupérer les meilleurs chunks
        best_chunks = df_chunks.iloc[idx[0]]["chunk"].tolist()
        prompt = user_query + "\n\n" + "\n".join(best_chunks)
        
        print("🔍 Prompt augmenté générée :\n", prompt)

        # ✅ Appel OpenAI corrigé
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}]
        )

        return response.choices[0].message.content if response.choices else "⚠️ Aucune réponse générée."

    except Exception as e:
        return f"⚠️ Erreur lors de la récupération : {str(e)}"


# 🔥 Test avec la question sur Lincoln
question = "What did Lincoln argue about slavery in that first debate?"
answer = retrieve(question)
print("🔍 Réponse générée :\n", answer)


### Bonnes pratiques pour la RAG (Retrieval Augmented Generation)

1. **Chunking**  
   - Découper vos documents en segments de taille raisonnable (ex. 300-500 tokens), afin de mieux cibler les passages pertinents.
2. **Indexation des embeddings**  
   - Stocker les vecteurs dans une base adaptée (ex. Pinecone, Chroma, Elasticsearch vectoriel, Qdrant, ou Cosmos DB vector).
3. **Filtrage et post-traitement**  
   - Après avoir récupéré les chunks les plus proches sémantiquement, il peut être utile de vérifier l’exactitude ou la cohérence des informations extraites.
4. **Ré-intégration**  
   - Insérer les passages sélectionnés dans le prompt (par ex. “Voici un extrait : ...\n\n Maintenant, réponds à la question...”).  
   - Ou bien utiliser un outil style [LangChain](https://github.com/hwchase17/langchain) qui facilite ce pipeline.
5. **Éviter les hallucinations**  
   - Demander explicitement au modèle de s’en tenir aux informations fournies dans les chunks.  
   - En cas d’insuffisance de données, demander au modèle de répondre “Je ne sais pas” plutôt que d’inventer.

L’objectif est de combiner la **puissance du LLM** et l’**exactitude** de données externes (base documentaire, articles, PDF, etc.).


# 5. Conclusion & Pistes

Nous avons exploré :
- la génération d’images (DALL-E, prompts, meta-prompts),
- la création d’apps low-code Power Apps / Automate,
- l’usage de Copilot & AI Builder pour des scénarios métiers (tracking, invoice),
- la structuration des réponses via Function Calling,
- RAG : indexer nos docs dans une base vectorielle et enrichir un LLM.

Pistes d’exercices :
- Améliorer les prompts d’images (température, variations, mask, etc.)
- Créer un flux complet dans Power Automate avec AI Builder
- Mettre en place Function Calling plus complexe (multi-fonctions, error-handling)
- Stocker un doc plus large (ex: 10 pages PDF) en chunks + RAG

Fin de la synthèse ! 
